---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    town = pd.read_table('university_towns.txt',header=None,names=['RegionName'])
    findstate = town['RegionName'].str.contains('[edit]',regex=False)
    town['State'] = town[findstate]
    town['State'] = town['State'].str.replace(r'(\[.*\])','')
    town['State'].fillna(method="ffill",inplace=True)
    town['RegionName'] = town['RegionName'].str.replace(r'\[.*\]','')
    town['RegionName'] = town['RegionName'].str.replace(r'\s\(.*\)','')
    town = town[~findstate].reset_index()
    newdata = pd.DataFrame(town,columns=['State','RegionName'])
    return newdata
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


In [140]:
gdp = pd.read_excel('gdplev.xls',header=5,usecols=[4,5,6],names=['Quarter', 'GDP current', 'GDP chained 2009']).dropna().reset_index(drop=True)
gdp = pd.concat([gdp,gdp['Quarter'].str.split('q',expand=True)],axis=1)
# gdp.drop('Quarter',axis=1,inplace=True)
gdp.columns = ['Quarter','GDP current', 'GDP chained 2009','Year','Q']
gdp['Year'] = gdp['Year'].apply(int)
gdp['Q'] = gdp['Q'].apply(int)
# gdp = gdp[gdp['Year']>2000].reset_index(drop=True)
gdp.set_index('Quarter',inplace=True)
print(gdp)

def cal_recession():
    result = {
        'start': [],
        'end': [],
        'bottom': []
    }
    
    down_counter = 2
    down_start_index = ''
    
    up_counter = 0
    
    bottom = 9999999
    bottom_index = ''
    
    flag = False
    
    last_value = gdp['GDP chained 2009'].iloc[0]
    print('initial value: {}'.format(last_value))
    
    for index, item in gdp.iterrows():
        value = item['GDP chained 2009']
        print('{} {}'.format(index, value))
        
        if value < last_value:
            print('------------ Begin to decrease ----------')
            if down_counter == 2:
                down_start_index = index
            down_counter -= 1
            if value < bottom:
                bottom = value
                bottom_index = index
        else:
            down_counter = 2
        
        if down_counter == 0:
            print('--------- Recession is coming ----------')
            bottom = value
            bottom_index = index
            flag = True
            up_counter = 0
            result['start'].append(down_start_index)
        
        if flag:
            if value > last_value:
                print('+++++++++++++ Begin to increase ++++++++++++')
                up_counter += 1
            else:
                up_counter = 0
                
            if up_counter ==2:
                print('+++++++++++++ Recession ends ++++++++++++++')
                down_counter = 0
                flag = False
                result['end'].append(index)
                result['bottom'].append(bottom_index)
                
        last_value = value
            
    return result
cal_recession()    
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    start = result['start']
    return start[-1]
get_recession_start()

         GDP current  GDP chained 2009  Year  Q
Quarter                                        
1947q1         243.1            1934.5  1947  1
1947q2         246.3            1932.3  1947  2
1947q3         250.1            1930.3  1947  3
1947q4         260.3            1960.7  1947  4
1948q1         266.2            1989.5  1948  1
...              ...               ...   ... ..
2015q2       17998.3           16374.2  2015  2
2015q3       18141.9           16454.9  2015  3
2015q4       18222.8           16490.7  2015  4
2016q1       18281.6           16525.0  2016  1
2016q2       18450.1           16583.1  2016  2

[278 rows x 4 columns]
initial value: 1934.5
1947q1 1934.5
1947q2 1932.3
------------ Begin to decrease ----------
1947q3 1930.3
------------ Begin to decrease ----------
--------- Recession is coming ----------
1947q4 1960.7
+++++++++++++ Begin to increase ++++++++++++
1948q1 1989.5
+++++++++++++ Begin to increase ++++++++++++
+++++++++++++ Recession ends ++++++++++++++
1

'2008q3'

In [141]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    end = result['end']   
    return end[-1]
get_recession_end()

'2009q4'

In [142]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    bottom = result['bottom']
    return bottom[-1]
get_recession_bottom()

'2009q2'

In [143]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing = pd.read_csv('City_Zhvi_AllHomes.csv',header=None)
    housing = housing.iloc[:,1:3].merge(housing.iloc[:,51:],left_index=True,right_index=True)
    housing = housing.T
    housing.reset_index(drop=True,inplace=True)
    housing[0].iloc[2:] = pd.to_datetime(housing[0].iloc[2:]).dt.to_period('q')
    housing[0].iloc[2:] = housing[0].iloc[2:].apply(lambda x: str(x).replace('Q','q'))
    housing = housing.T.set_index(0).T
    housing = housing.set_index('RegionName').T
    housing.index.names=['RegionName']
    housing.columns.names=['']
    housing=housing.apply(lambda x: pd.to_numeric(x, errors='ignore'))
    h = housing.iloc[:,2:]
    h = h.groupby(level=0,axis=1).mean()
    housing['State'] = housing['State'].replace(states)
    housing.reset_index(inplace=True)
    
    return h.set_index([housing['State'],housing['RegionName']])
convert_housing_data_to_quarters()

C:\Users\guotree\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (0,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250) have m

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,208400.0,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,139200.0,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53150.0,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,112250.0,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Town of Wrightstown,102100.0,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
New York,Urbana,80000.0,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
Wisconsin,New Denmark,114900.0,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0


In [144]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    housing = convert_housing_data_to_quarters()
    price = pd.DataFrame(housing[get_recession_bottom()]-housing[get_recession_start()],columns=['delta_price'])
    university_towns = get_list_of_university_towns()
    university_towns['test'] = university_towns['State'] + university_towns['RegionName']
    price = price.reset_index()
    price['Is_utowns'] = price.apply(lambda x: x[0]+x[1] in list(university_towns['test']),axis=1)
    price_u = price[price['Is_utowns']]['delta_price']
    price_nu = price[~price['Is_utowns']]['delta_price']
    statistic, pvalue = ttest_ind(price_u,price_nu,nan_policy='omit')
    if pvalue < 0.01:
        different = True
    else:
        different = False
    if price_u.mean() > price_nu.mean():
        better = 'university town'
    else:
        better = 'non-university town'
    return (different, pvalue, better)
run_ttest()

C:\Users\guotree\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (0,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250) have m

(True, 0.0025826404310383827, 'university town')